In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import numpy as np

In [2]:
import os
import cv2

In [3]:
import pandas as pd

In [4]:
sources = ["https://towardsdatascience.com/build-your-own-convolution-neural-network-in-5-mins-4217c2cf964f", "https://arxiv.org/pdf/1409.1556.pdf"]

In [5]:
# Image Dimension
H = 128
W = 128
C = 1

In [22]:
def CNN_EXP(num_classes):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(H,W,1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='Adagrad',metrics=['accuracy'])
    return model

In [6]:
def CNN_VGG_Lite(num_classes):
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(3,3),padding='same',
                     activation='relu', input_shape=(H,W,C)))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
    model.add(Conv2D(128, (3, 3), padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
    model.add(Conv2D(256, (3, 3), padding='same',activation='relu'))
    model.add(Conv2D(256, (3, 3), padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd',metrics=['accuracy'])
    return model

# JAFFE utilization

In [6]:
image_root_jaffe = 'dataset/jaffe/'
image_title_jaffe = os.listdir(image_root)
image_dirs_jaffe = [image_root + x for x in os.listdir(image_root)]

# AffectNet utilization

In [8]:
cd D:\Documents\KULIAH\DSA\TA\Manually_Annotated

D:\Documents\KULIAH\DSA\TA\Manually_Annotated


In [9]:
training_csv = pd.read_csv('training.csv')

(0: Neutral, 1: Happy, 2: Sad, 3:
Surprise, 4: Fear, 5: Disgust, 6: Anger, 7: Contempt, 8: None, 9:
Uncertain, 10: No-Face)

In [10]:
validation_csv = pd.read_csv('validation.csv')

In [11]:
image_root = 'Manually_Annotated_Images\\'
image_dirs = []

In [12]:
training_csv.head()

,subDirectory_filePath,face_x,face_y,face_width,face_height,facial_landmarks,expression,valence,arousal
0,689/737db2483489148d783ef278f43f486c0a97e140fc...,134,134,899,899,181.64;530.91;188.32;627.82;195.1;723.37;205.2...,1,0.785714,-0.055556
1,392/c4db2f9b7e4b422d14b6e038f0cdc3ecee239b5532...,20,20,137,137,28.82;77.52;29.12;93.25;31.04;108.51;33.03;123...,0,-0.017253,0.004313
2,468/21772b68dc8c2a11678c8739eca33adb6ccc658600...,11,11,176,176,30.52;87.33;32.55;106.43;36.94;125.81;43.06;14...,0,0.174603,0.007937
3,944/06e9ae8d3b240eb68fa60534783eacafce2def60a8...,40,40,269,269,44.43;158.17;47.08;189.2;50.54;221.88;58.3;253...,1,0.153401,0.038890
4,993/02e06ee5521958b4042dd73abb444220609d96f57b...,22,22,153,153,50.59;78.72;48.6;93.23;48.72;109.06;48.8;123.0...,8,0.783972,-0.551684


In [13]:
size = 30000

In [14]:
image_dirs = []
for i in range(size):
    path = image_root + training_csv['subDirectory_filePath'][i].replace('/','\\')
    image_dirs.append(path)

In [15]:
def read_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    dimension = (H,W)
    resized = cv2.resize(img, dimension)
    return resized
def prep_data(images):
    count = len(images)
    data = np.ndarray((count, C, H, W), dtype=np.uint8)
    for i, image_file in enumerate(images):
        image = read_image(image_file)
        data[i] = image
        if i%1000 == 0: print('Processed {} of {}'.format(i, count))
    return data

In [16]:
images = prep_data(image_dirs)

Processed 0 of 30000
Processed 1000 of 30000
Processed 2000 of 30000
Processed 3000 of 30000
Processed 4000 of 30000
Processed 5000 of 30000
Processed 6000 of 30000
Processed 7000 of 30000
Processed 8000 of 30000
Processed 9000 of 30000
Processed 10000 of 30000
Processed 11000 of 30000
Processed 12000 of 30000
Processed 13000 of 30000
Processed 14000 of 30000
Processed 15000 of 30000
Processed 16000 of 30000
Processed 17000 of 30000
Processed 18000 of 30000
Processed 19000 of 30000
Processed 20000 of 30000
Processed 21000 of 30000
Processed 22000 of 30000
Processed 23000 of 30000
Processed 24000 of 30000
Processed 25000 of 30000
Processed 26000 of 30000
Processed 27000 of 30000
Processed 28000 of 30000
Processed 29000 of 30000


In [17]:
# for AffectNet
labels = training_csv['expression'][:size]

In [ ]:
# For jaffe
image_labels_jaffe = np.array([title[3:5] for title in image_title])
labels_jaffe = []
for i, label in enumerate(image_labels_jaffe):
    if label == 'AN':
        labels_jaffe.append(0)
    if label == 'DI':
        labels_jaffe.append(1)
    if label == 'FE':
        labels_jaffe.append(2)
    if label == 'SU':
        labels_jaffe.append(3)
    if label == 'SA':
        labels_jaffe.append(4)
    if label == 'HA':
        labels_jaffe.append(5)
    if label == 'NE':
        labels_jaffe.append(6)

In [30]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
X_train = images[:8000]
X_test = images[8000:10000]

X_train = X_train.reshape(X_train.shape[0], H, W, C)
X_test = X_test.reshape(X_test.shape[0], H, W, C)


y_train = labels[:8000]
y_test = labels[8000:10000]


In [31]:
cnn_exp = CNN_EXP(11)

In [32]:
cnn_exp.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 126, 126, 32)      320       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 124, 124, 64)      18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 41, 41, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 41, 41, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 107584)            0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               13770880  
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
__________

In [33]:
cnn_exp.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10,batch_size=8, verbose=2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
 - 23s - loss: 11.2034 - acc: 0.3047 - val_loss: 11.0490 - val_acc: 0.3145
Epoch 2/10
 - 23s - loss: 11.1275 - acc: 0.3096 - val_loss: 11.0490 - val_acc: 0.3145
Epoch 3/10
 - 23s - loss: 10.9805 - acc: 0.3187 - val_loss: 11.0490 - val_acc: 0.3145
Epoch 4/10
 - 23s - loss: 10.9704 - acc: 0.3194 - val_loss: 11.0490 - val_acc: 0.3145
Epoch 5/10
 - 23s - loss: 11.0454 - acc: 0.3146 - val_loss: 11.0490 - val_acc: 0.3145
Epoch 6/10
 - 23s - loss: 11.6937 - acc: 0.2745 - val_loss: 11.0490 - val_acc: 0.3145
Epoch 7/10
 - 23s - loss: 11.7984 - acc: 0.2680 - val_loss: 11.0490 - val_acc: 0.3145
Epoch 8/10
 - 23s - loss: 11.5929 - acc: 0.2808 - val_loss: 11.0490 - val_acc: 0.3145
Epoch 9/10
 - 23s - loss: 11.5667 - acc: 0.2824 - val_loss: 11.0490 - val_acc: 0.3145
Epoch 10/10
 - 23s - loss: 11.6332 - acc: 0.2783 - val_loss: 11.0490 - val_acc: 0.3145


In [34]:
scores = cnn_exp.evaluate(X_test, y_test, verbose=0)
print("Classification Error: %.2f%%" % (100-scores[1]*100))

Classification Error: 68.55%


In [27]:
model_vgg_lite.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 64)      640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 63, 63, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 31, 31, 256)       295168    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 31, 31, 256)       590080    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 256)       0         
__________

In [19]:
model_vgg_lite.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20,batch_size=8, verbose=2)

Train on 28000 samples, validate on 2000 samples
Epoch 1/20
 - 106s - loss: 13.1882 - acc: 0.1817 - val_loss: 13.6117 - val_acc: 0.1555
Epoch 2/20
 - 91s - loss: 13.1886 - acc: 0.1817 - val_loss: 13.6117 - val_acc: 0.1555
Epoch 3/20
 - 91s - loss: 13.1886 - acc: 0.1817 - val_loss: 13.6117 - val_acc: 0.1555
Epoch 4/20
 - 91s - loss: 13.1886 - acc: 0.1817 - val_loss: 13.6117 - val_acc: 0.1555
Epoch 5/20
 - 91s - loss: 13.1886 - acc: 0.1817 - val_loss: 13.6117 - val_acc: 0.1555
Epoch 6/20
 - 91s - loss: 13.1886 - acc: 0.1817 - val_loss: 13.6117 - val_acc: 0.1555
Epoch 7/20
 - 91s - loss: 13.1886 - acc: 0.1817 - val_loss: 13.6117 - val_acc: 0.1555
Epoch 8/20
 - 91s - loss: 13.1886 - acc: 0.1817 - val_loss: 13.6117 - val_acc: 0.1555
Epoch 9/20
 - 91s - loss: 13.1886 - acc: 0.1817 - val_loss: 13.6117 - val_acc: 0.1555
Epoch 10/20
 - 91s - loss: 13.1886 - acc: 0.1817 - val_loss: 13.6117 - val_acc: 0.1555
Epoch 11/20
 - 91s - loss: 13.1886 - acc: 0.1817 - val_loss: 13.6117 - val_acc: 0.1555
Ep

In [20]:
scores = model_vgg_lite.evaluate(X_test, y_test, verbose=0)
print("Classification Error: %.2f%%" % (100-scores[1]*100))

Classification Error: 84.45%


In [7]:
model_vgg_lite = CNN_VGG_Lite(11)

Instructions for updating:
Colocations handled automatically by placer.


In [35]:
cnn_exp.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10,batch_size=15, verbose=2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
 - 17s - loss: 11.5869 - acc: 0.2811 - val_loss: 11.0490 - val_acc: 0.3145
Epoch 2/10
 - 17s - loss: 12.6525 - acc: 0.2149 - val_loss: 13.0234 - val_acc: 0.1920
Epoch 3/10
 - 17s - loss: 12.8351 - acc: 0.2036 - val_loss: 13.0234 - val_acc: 0.1920
Epoch 4/10
 - 17s - loss: 12.7131 - acc: 0.2113 - val_loss: 13.0234 - val_acc: 0.1920
Epoch 5/10
 - 17s - loss: 12.6406 - acc: 0.2157 - val_loss: 13.0234 - val_acc: 0.1920
Epoch 6/10
 - 17s - loss: 12.7796 - acc: 0.2071 - val_loss: 13.0234 - val_acc: 0.1920
Epoch 7/10
 - 17s - loss: 12.6628 - acc: 0.2144 - val_loss: 13.0234 - val_acc: 0.1920
Epoch 8/10
 - 17s - loss: 12.7655 - acc: 0.2080 - val_loss: 13.0234 - val_acc: 0.1920
Epoch 9/10
 - 17s - loss: 12.7172 - acc: 0.2110 - val_loss: 13.0234 - val_acc: 0.1920
Epoch 10/10
 - 17s - loss: 12.7112 - acc: 0.2114 - val_loss: 13.0234 - val_acc: 0.1920


In [36]:
scores = cnn_exp.evaluate(X_test, y_test, verbose=0)
print("Classification Error: %.2f%%" % (100-scores[1]*100))

Classification Error: 80.80%
